
** 作业5 **

- 要求：

1. 利用 TensorFlow Recommenders（TFRS）API 实现双塔模型


2. 基于 MovieLens 数据集训练模型（对于电影，需要用除了 ID 以外的特征训练模型）


---



In [ ]:
import os
import pprint
import tempfile
from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs


这些是导入的相关包。



1. 数据加载与准备

In [ ]:
# 加载 MovieLens 100k 数据集
print("Loading dataset...")
# 在这里，我们保留 movie_genres 的原始整数格式
ratings = tfds.load("movielens/100k-ratings", split="train")
movies = tfds.load("movielens/100k-movies", split="train")

# 提取用户ID和电影特征。注意这里 movie_genres 保持不变。
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "movie_genres": x["movie_genres"]
})

movies = movies.map(lambda x: {
    "movie_title": x["movie_title"],
    "movie_genres": x["movie_genres"]
})

# 构建用户ID和电影标题的词汇表 (它们是字符串/bytes)
print("Building vocabularies for user_id and movie_title...")
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies.map(lambda x: x["movie_title"]))

# --- 找出电影类型的最大 ID ---
# 因为 movie_genres 已经是整数 ID，我们需要找到最大的 ID 来设置 Embedding 层的大小
print("Finding maximum movie genre ID...")
all_movie_genres_ids = []
for movie in movies.map(lambda x: x["movie_genres"]):
    # movie["movie_genres"] 是一个张量，可能包含多个 genre ID
    # 在这里直接处理原始 movies 数据集，它里面的 movie_genres 仍然是标准的tf.Tensor
    all_movie_genres_ids.extend(movie.numpy().tolist()) # 将张量转换为列表并添加到总列表

max_movie_genre_id = max(all_movie_genres_ids)
num_movie_genres = max_movie_genre_id + 1 # Embedding 层维度需要包含所有可能的 ID，从 0 到 max_id

print(f"Maximum movie genre ID found: {max_movie_genre_id}")
print(f"Number of unique movie genres (for embedding layer): {num_movie_genres}")


# 数据划分
print("Splitting data into train and test sets...")
tf.random.set_seed(42)
shuffled_ratings = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled_ratings.take(80_000)
test = shuffled_ratings.skip(80_000).take(20_000)

# 批量处理和缓存数据
print("Batching and caching data...")
# 修改：使用 ragged_batch 来处理训练/测试数据中 movie_genres 的可变长度
cached_train = train.ragged_batch(4096).cache()
cached_test = test.ragged_batch(4096).cache()

2. 模型构建 (双塔模型)


In [ ]:

embedding_dimension = 64  # 嵌入维度，可以调整

# 定义用户模型 (Query tower)
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    # 修正：使用 vocabulary_size()
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), embedding_dimension)
])

# 定义电影模型 (Candidate tower)
class MovieCandidateModel(tf.keras.Model):
    def __init__(self, movie_titles_vocabulary, num_movie_genres, embedding_dimension):
        super().__init__()
        # 处理电影标题 (字符串 -> 嵌入)
        self.movie_titles_embedding = tf.keras.Sequential([
            movie_titles_vocabulary,
            # 修正：使用 vocabulary_size()
            tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), embedding_dimension)
        ])
        # 处理电影类型 (整数 ID -> 嵌入)
        # Embedding 层能够处理 RaggedTensor 输入
        self.movie_genres_embedding = tf.keras.layers.Embedding(
            input_dim=num_movie_genres,
            output_dim=embedding_dimension
        )
        # 组合不同特征的 embeddings
        self.combination_layer = tf.keras.layers.Dense(embedding_dimension, activation="relu")

    def call(self, inputs):
        # inputs["movie_title"] 是一个 Tensor 或 RaggedTensor (来自 ragged_batch)
        # inputs["movie_genres"] 是一个 RaggedTensor (来自 ragged_batch)
        title_embedding = self.movie_titles_embedding(inputs["movie_title"])
        genres_embedding = self.movie_genres_embedding(inputs["movie_genres"]) # Output is RaggedTensor

        # 对 RaggedTensor 进行求和，axis=1 会在每个样本内部的第二个维度（嵌入维度）求和
        # 结果 shape 是 [batch_size, embedding_dimension]
        genres_embedding_aggregated = tf.reduce_sum(genres_embedding, axis=1)

        # 确保 title_embedding 是密集张量以便拼接
        # 理论上 movie_title 应该是密集张量，但为了保险，这里检查并转换
        if isinstance(title_embedding, tf.RaggedTensor):
            title_embedding = title_embedding.to_tensor()


        combined_embedding = tf.concat([title_embedding, genres_embedding_aggregated], axis=1)
        return self.combination_layer(combined_embedding)


# 定义完整的 TFRS 模型
class MovieLensModel(tfrs.Model):
    def __init__(self, user_model, movie_model, movies_dataset):
        super().__init__()
        self.user_model = user_model
        self.movie_model = movie_model
        # 定义检索任务和评估指标
        # 注意：这里的 candidates 应该是经过 movie_model 处理后的电影 embeddings
        # 这里需要将 movies_dataset 的每个元素（字典）传递给 movie_model
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                # 修改：对用于 candidates 的数据集也使用 ragged_batch
                candidates=movies_dataset.ragged_batch(128).map(self.movie_model)
            )
        )

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # features["user_id"] 是一个 Tensor (通常来自原始数据集或 map)
        # features["movie_title"] 是一个 Tensor 或 RaggedTensor (来自 ragged_batch)
        # features["movie_genres"] 是一个 RaggedTensor (来自 ragged_batch)

        # 计算用户和电影的 embeddings
        user_embeddings = self.user_model(features["user_id"])
        # 将完整的电影特征字典传递给 movie_model
        movie_embeddings = self.movie_model({
            "movie_title": features["movie_title"],
            "movie_genres": features["movie_genres"]
        })

        # 将 embeddings 传递给任务计算损失
        return self.task(user_embeddings, movie_embeddings)

# 创建电影候选模型实例
movie_candidate_model = MovieCandidateModel(movie_titles_vocabulary, num_movie_genres, embedding_dimension)

# 创建完整的 TFRS 模型实例
# 注意：这里的 movies 数据集需要保留 movie_genres 特征以便传递给 movie_model
model = MovieLensModel(user_model, movie_candidate_model, movies)


3. 模型编译与训练

In [ ]:



print("Compiling and training the model...")
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# 训练模型
model.fit(cached_train, epochs=3) # 可以调整训练轮数



4. 模型评估



In [ ]:
print("Evaluating the model...")
model.evaluate(cached_test, return_dict=True)


5. 生成推荐

In [ ]:

print("Generating recommendations...")
# 构建检索索引。索引是基于电影 embeddings 构建的。
# 注意：这里需要使用 movie_candidate_model 来处理电影数据以获取 embeddings
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# 在 index_from_dataset 中，需要传递电影的特征字典给 movie_model
# 修改：对用于构建索引的 movies 数据集也使用 ragged_batch
index.index_from_dataset(
    movies.ragged_batch(100).map(lambda movie_features: (
        movie_features["movie_title"], # 使用电影标题作为标识符
        model.movie_model(movie_features) # 将电影特征字典传递给 movie_model
    ))
)

# 为特定用户生成 Top-K 推荐
user_id_to_recommend = "33"
# 获取用户 embedding
# user_embedding = model.user_model(tf.constant([user_id_to_recommend])) # 这行不再需要了

# 使用索引查找最相似的电影
# index 返回两个张量：第一个是相似度得分，第二个是电影标题（或其他你用于索引的标识符）
# 修改：直接将原始用户 ID 张量传递给 index 层，它会内部调用 user_model
scores, titles = index(tf.constant([user_id_to_recommend])) # 修改为这行

print(f"Top recommendations for user {user_id_to_recommend}:")
# 解码 byte string 并打印电影标题
for title in titles[0, :10].numpy():
    print(title.decode("utf-8"))

总体解释

数据加载与准备: 从 tensorflow_datasets 中加载 MovieLens 100k 数据集，提取出用户ID、电影标题和电影类型等特征。对用户ID和电影标题构建词汇表，并找出电影类型ID的最大值以确定 Embedding 层的维度。将数据划分为训练集和测试集，并使用 ragged_batch 进行批量处理和缓存，以处理电影类型列表的可变长度。

模型构建: 定义了两个独立的神经网络模型：用户模型（Query tower）和电影模型（Candidate tower）。用户模型将用户ID嵌入到低维向量空间。电影模型将电影标题（通过词汇表和 Embedding）和电影类型（通过 Embedding 并求和）组合起来，嵌入到低维向量空间。然后，定义了一个完整的 TFRS MovieLensModel，它结合了用户模型和电影模型，并使用 tfrs.tasks.Retrieval 定义了召回任务和 FactorizedTopK 评估指标。

模型编译与训练: 使用 Adagrad 优化器编译模型，然后在处理好的训练数据集上进行训练。

模型评估: 在处理好的测试数据集上评估模型的性能，使用 FactorizedTopK 指标来衡量召回的准确性。

生成推荐: 构建一个基于电影 embeddings 的检索索引 (BruteForce)。然后，为特定用户（例如用户ID "42" 或 "33"）计算其 embedding，并使用索引查找与其 embedding 最相似的 Top-K 部电影，最后输出推荐的电影标题。

简而言之，代码的核心思想是将用户和电影都表示成低维度的向量（embeddings），并通过训练使得用户 embedding 与用户喜欢（评分高）的电影 embedding 在向量空间中距离更近。在推荐时，只需找到用户 embedding 最接近的电影 embeddings 对应的电影。



运行结果




In [ ]:
D:\devenvironment\Anaconda\anacondadata\envs\py310\python.exe C:\Users\qiqi\OneDrive\Desktop\code\pythonweb\t5\untitled\1.py
2025-05-09 18:51:25.218567: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
WARNING:tensorflow:From D:\devenvironment\Anaconda\anacondadata\envs\py310\lib\site-packages\keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.

Loading dataset...
2025-05-09 18:51:30.361062: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE SSE2 SSE3 SSE4.1 SSE4.2 AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Building vocabularies for user_id and movie_title...
WARNING:tensorflow:From D:\devenvironment\Anaconda\anacondadata\envs\py310\lib\site-packages\keras\src\backend.py:873: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

WARNING:tensorflow:From D:\devenvironment\Anaconda\anacondadata\envs\py310\lib\site-packages\keras\src\backend.py:873: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

WARNING:tensorflow:From D:\devenvironment\Anaconda\anacondadata\envs\py310\lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.

WARNING:tensorflow:From D:\devenvironment\Anaconda\anacondadata\envs\py310\lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.

Finding maximum movie genre ID...
Maximum movie genre ID found: 19
Number of unique movie genres (for embedding layer): 20
Splitting data into train and test sets...
                                   Batching and caching data...
                                                        Compiling and training the model...
Epoch 1/3
20/20 [==============================] - 6s 228ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0281 - factorized_top_k/top_5_categorical_accuracy: 0.1985 - factorized_top_k/top_10_categorical_accuracy: 0.4076 - factorized_top_k/top_50_categorical_accuracy: 0.7261 - factorized_top_k/top_100_categorical_accuracy: 0.7475 - loss: 59228.3921 - regularization_loss: 0.0000e+00 - total_loss: 59228.3921
Epoch 2/3
20/20 [==============================] - 4s 216ms/step - factorized_top_k/top_1_categorical_accuracy: 0.2129 - factorized_top_k/top_5_categorical_accuracy: 0.9353 - factorized_top_k/top_10_categorical_accuracy: 0.9996 - factorized_top_k/top_50_categorical_accuracy: 0.9996 - factorized_top_k/top_100_categorical_accuracy: 0.9996 - loss: 32417.6583 - regularization_loss: 0.0000e+00 - total_loss: 32417.6583
Epoch 3/3
20/20 [==============================] - 4s 216ms/step - factorized_top_k/top_1_categorical_accuracy: 0.3454 - factorized_top_k/top_5_categorical_accuracy: 0.9998 - factorized_top_k/top_10_categorical_accuracy: 0.9998 - factorized_top_k/top_50_categorical_accuracy: 0.9998 - factorized_top_k/top_100_categorical_accuracy: 0.9998 - loss: 32417.5084 - regularization_loss: 0.0000e+00 - total_loss: 32417.5084
Evaluating the model...
5/5 [==============================] - 2s 195ms/step - factorized_top_k/top_1_categorical_accuracy: 0.3415 - factorized_top_k/top_5_categorical_accuracy: 0.9995 - factorized_top_k/top_10_categorical_accuracy: 0.9995 - factorized_top_k/top_50_categorical_accuracy: 0.9995 - factorized_top_k/top_100_categorical_accuracy: 0.9995 - loss: 32588.9043 - regularization_loss: 0.0000e+00 - total_loss: 32588.9043
Generating recommendations...
Top recommendations for user 33:
    Children of the Corn: The Gathering (1996)
You So Crazy (1994)
Love Is All There Is (1996)
Fly Away Home (1996)
In the Line of Duty 2 (1987)
Niagara, Niagara (1997)
Young Poisoner's Handbook, The (1995)
Age of Innocence, The (1993)
Flirt (1995)
Frisk (1995)

进程已结束，退出代码为 0


运行结果分析：

数据加载与准备: 脚本成功加载了 MovieLens 100k 数据集，构建了用户ID和电影标题的词汇表，确定了电影类型的数量，并将数据正确地划分并使用 ragged_batch 进行了批量处理和缓存。这些步骤都没有出现错误。

模型训练: 模型在训练集上进行了 3 个 epoch 的训练。从输出的指标可以看到：

loss（总损失）在每个 epoch 结束时逐渐下降，表明模型正在学习。
factorized_top_k/top_1_categorical_accuracy（Top-1 准确率）在训练过程中有所提升，从第一轮的约 0.0281 提高到第三轮的约 0.3454。这意味着在训练集上，模型预测得分最高的电影是用户实际交互过的电影的比例有所增加。
top_5、top_10、top_50 和 top_100 的准确率在训练后期迅速接近 1.0000。这表明虽然模型不一定总能将用户实际交互过的电影排在第一位，但它很有可能将这些电影排在预测结果的前几名或前几十名中。
模型评估: 脚本在测试集上对训练好的模型进行了评估。

factorized_top_k/top_1_categorical_accuracy 在测试集上约为 0.3415，与训练集上的最终结果接近。这表明模型具有一定的泛化能力，在未见过的数据上也能保持类似的 Top-1 性能。
同样，测试集上的其他 Top-K 准确率（Top-5 到 Top-100）也非常高，接近 1.0000。这再次强调了模型能够有效地将用户可能感兴趣的电影包含在 Top-K 的推荐列表中。
生成推荐: 脚本成功地为用户 33 生成了 Top-10 推荐电影列表，并打印出了电影标题。这验证了构建的检索索引和推荐流程是正常工作的。

总结:

总体而言，运行结果显示你已经成功地使用 TFRS 库构建并训练了一个基本的双塔召回模型，并且能够用它来为用户生成推荐。训练和评估指标看起来合理，特别是较高的 Top-K 准确率符合召回模型的特点（旨在从大量候选集中召回用户可能感兴趣的一小部分）。